In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
import pickle
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import NMF
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.metrics import average_precision_score


In [ ]:
#En yüksek doğruluk vermesi için verinin boşlukları MICE yöntemi ile doldurma,Ayrıca Son Arayüzde Kullandığımız veriyi oluşturma
def veriseti_duzenleme():
    df=pd.read_csv('US_Accidents_June20.csv')
    feature_list=['Start_Time','End_Time','Wind_Chill(F)','TMC','Severity','Wind_Speed(mph)','Distance(mi)','Side','Precipitation(in)',                             'State',
                      'Temperature(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Direction',
                      'Weather_Condition','Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway',
                      'Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Sunrise_Sunset','Civil_Twilight',
                      'Nautical_Twilight','Astronomical_Twilight']
    data=df[feature_list].copy()
    data['Wind_Chill(F)']=(data['Wind_Chill(F)']-32)*(5/9)+273.15
    data['Temperature(F)']=(data['Temperature(F)']-32)*(5/9)+273.15
    data['Amenity']=data['Amenity'].astype(int)
    data['Bump']=data['Bump'].astype(int)
    data['Crossing']=data['Crossing'].astype(int)
    data['Give_Way']=data['Give_Way'].astype(int)
    data['Junction']=data['Junction'].astype(int)
    data['No_Exit']=data['No_Exit'].astype(int)
    data['Railway']=data['Railway'].astype(int)
    data['Roundabout']=data['Roundabout'].astype(int)
    data['Station']=data['Station'].astype(int)
    data['Stop']=data['Stop'].astype(int)
    data['Traffic_Calming']=data['Traffic_Calming'].astype(int)
    data['Traffic_Signal']=data['Traffic_Signal'].astype(int)
    data.Side = [1 if each == "R" else 0 for each in data.Side]
    data.Sunrise_Sunset = [1 if each == "Day" else 0 for each in data.Sunrise_Sunset]
    data.Civil_Twilight = [1 if each == "Day" else 0 for each in data.Civil_Twilight]
    data.Nautical_Twilight = [1 if each == "Day" else 0 for each in data.Nautical_Twilight]
    data.Astronomical_Twilight = [1 if each == "Day" else 0 for each in data.Astronomical_Twilight]
    data.State=data.State.astype("category").cat.codes
    data['Start_Time'] = pd.to_datetime(data['Start_Time'], errors='coerce')
    data['End_Time'] = pd.to_datetime(data['End_Time'], errors='coerce')
    data['Year']=data['Start_Time'].dt.year
    data['Month']=data['Start_Time'].dt.strftime('%b')
    data['Day']=data['Start_Time'].dt.day
    data['Hour']=data['Start_Time'].dt.hour
    data['Weekday']=data['Start_Time'].dt.strftime('%a')
    td='Time_Duration(min)'
    data[td]=round((data['End_Time']-data['Start_Time'])/np.timedelta64(1,'m'))
    data = data[data['Time_Duration(min)'] >= 0]
    d = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
    data.Month = data.Month.map(d)
    w = {'Mon':1, 'Tue':2, 'Wed':3, 'Thu':4, 'Fri':5, 'Sat':6, 'Sun':7}
    data.Weekday = data.Weekday.map(w)
    data=data.drop('End_Time',axis=1)
    data=data.drop('Start_Time',axis=1)
    data.Wind_Direction=data.Wind_Direction.astype("category").cat.codes
    data.Weather_Condition=data.Weather_Condition.astype("category").cat.codes
    lr = LinearRegression()
    imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=5)
    imp_data=imp.fit_transform(data)
    df = pd.DataFrame(data=imp_data, index=data.index, columns=data.columns)
    return df

In [ ]:
# VERİSETİNDEKİ BOŞLUKLARI ORTALAMA DEĞER İLE DOLDURMA 

def veriseti_duzenleme_ortalama():
    df=pd.read_csv('US_Accidents_June20.csv')
    feature_list=['Start_Time','End_Time','Wind_Chill(F)','TMC','Severity','Wind_Speed(mph)','Distance(mi)','Side','Precipitation(in)',                         'State',
                  'Temperature(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Direction',
                  'Weather_Condition','Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway',
                  'Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Sunrise_Sunset','Civil_Twilight',
                  'Nautical_Twilight','Astronomical_Twilight']
    data=df[feature_list].copy()
    data['Wind_Chill(F)']=(data['Wind_Chill(F)']-32)*(5/9)+273.15
    data['Temperature(F)']=(data['Temperature(F)']-32)*(5/9)+273.15
    data['Wind_Chill(F)'] = data['Wind_Chill(F)'].fillna((data['Wind_Chill(F)'].mean()))
    data['Wind_Speed(mph)'] = data['Wind_Speed(mph)'].fillna((data['Wind_Speed(mph)'].mean()))
    data['TMC'] = data['TMC'].fillna((data['TMC'].mean()))
    data['Precipitation(in)'] = data['Precipitation(in)'].fillna((data['Precipitation(in)'].mean()))
    data['Temperature(F)'] = data['Temperature(F)'].fillna((data['Temperature(F)'].mean()))
    data['Humidity(%)'] = data['Humidity(%)'].fillna((data['Humidity(%)'].mean()))
    data['Pressure(in)'] = data['Pressure(in)'].fillna((data['Pressure(in)'].mean()))
    data['Visibility(mi)'] = data['Visibility(mi)'].fillna((data['Visibility(mi)'].mean()))
    data['Amenity']=data['Amenity'].astype(int)
    data['Bump']=data['Bump'].astype(int)
    data['Crossing']=data['Crossing'].astype(int)
    data['Give_Way']=data['Give_Way'].astype(int)
    data['Junction']=data['Junction'].astype(int)
    data['No_Exit']=data['No_Exit'].astype(int)
    data['Railway']=data['Railway'].astype(int)
    data['Roundabout']=data['Roundabout'].astype(int)
    data['Station']=data['Station'].astype(int)
    data['Stop']=data['Stop'].astype(int)
    data['Traffic_Calming']=data['Traffic_Calming'].astype(int)
    data['Traffic_Signal']=data['Traffic_Signal'].astype(int)
    data.Side = [1 if each == "R" else 0 for each in data.Side]
    data.Sunrise_Sunset = [1 if each == "Day" else 0 for each in data.Sunrise_Sunset]
    data.Civil_Twilight = [1 if each == "Day" else 0 for each in data.Civil_Twilight]
    data.Nautical_Twilight = [1 if each == "Day" else 0 for each in data.Nautical_Twilight]
    data.Astronomical_Twilight = [1 if each == "Day" else 0 for each in data.Astronomical_Twilight]
    data.State=data.State.astype("category").cat.codes
    data['Start_Time'] = pd.to_datetime(data['Start_Time'], errors='coerce')
    data['End_Time'] = pd.to_datetime(data['End_Time'], errors='coerce')
    data['Year']=data['Start_Time'].dt.year
    data['Month']=data['Start_Time'].dt.strftime('%b')
    data['Day']=data['Start_Time'].dt.day
    data['Hour']=data['Start_Time'].dt.hour
    data['Weekday']=data['Start_Time'].dt.strftime('%a')
    td='Time_Duration(min)'
    data[td]=round((data['End_Time']-data['Start_Time'])/np.timedelta64(1,'m'))
    data = data[data['Time_Duration(min)'] >= 0]
    d = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
    data.Month = data.Month.map(d)
    w = {'Mon':1, 'Tue':2, 'Wed':3, 'Thu':4, 'Fri':5, 'Sat':6, 'Sun':7}
    data.Weekday = data.Weekday.map(w)
    data=data.drop('End_Time',axis=1)
    data=data.drop('Start_Time',axis=1)
    data=data.dropna()
    data.Wind_Direction = data.Wind_Direction.astype("category").cat.codes
    data.Weather_Condition = data.Weather_Condition.astype("category").cat.codes
    return data

In [ ]:
# VERİSETİNDEKİ BOŞLUKLARI SİLEREK VERİSETİNİ DÜZENLEME

# def veriseti_duzenleme():
#     df=pd.read_csv('US_Accidents_June20.csv')
#     feature_list=['Wind_Chill(F)','TMC','Severity','Wind_Speed(mph)','Distance(mi)','Side','Precipitation(in)','State',
#                   'Temperature(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Direction',
#                   'Weather_Condition','Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway',
#                   'Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Sunrise_Sunset','Civil_Twilight',
#                   'Nautical_Twilight','Astronomical_Twilight']
#     data=df[feature_list].copy()
#     data['Wind_Chill(F)']=(data['Wind_Chill(F)']-32)*(5/9)+273.15
#     data['Temperature(F)']=(data['Temperature(F)']-32)*(5/9)+273.15
#     data=data.dropna()
#     data['Amenity']=data['Amenity'].astype(int)
#     data['Bump']=data['Bump'].astype(int)
#     data['Crossing']=data['Crossing'].astype(int)
#     data['Give_Way']=data['Give_Way'].astype(int)
#     data['Junction']=data['Junction'].astype(int)
#     data['No_Exit']=data['No_Exit'].astype(int)
#     data['Railway']=data['Railway'].astype(int)
#     data['Roundabout']=data['Roundabout'].astype(int)
#     data['Station']=data['Station'].astype(int)
#     data['Stop']=data['Stop'].astype(int)
#     data['Traffic_Calming']=data['Traffic_Calming'].astype(int)
#     data['Traffic_Signal']=data['Traffic_Signal'].astype(int)
#     data.Side = [1 if each == "R" else 0 for each in data.Side]
#     data.Sunrise_Sunset = [1 if each == "Day" else 0 for each in data.Sunrise_Sunset]
#     data.Civil_Twilight = [1 if each == "Day" else 0 for each in data.Civil_Twilight]
#     data.Nautical_Twilight = [1 if each == "Day" else 0 for each in data.Nautical_Twilight]
#     data.Astronomical_Twilight = [1 if each == "Day" else 0 for each in data.Astronomical_Twilight]
#     data.Wind_Direction = data.Wind_Direction.astype("category").cat.codes
#     data.Weather_Condition = data.Weather_Condition.astype("category").cat.codes
#     data.State = data.State.astype("category").cat.codes
#     return data

In [ ]:
data=veriseti_duzenleme()
target='Severity'
Y=data[target]
X=data.drop(target,axis=1)
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
# precision_gr=np.zeros(5)      RAPORDA KULLANILAN GRAFİKLER İÇİN
# recall_gr=np.zeros(5)
# f1score_gr=np.zeros(5)
# accuracy_gr=np.zeros(5)
def decision_tree(x_train,y_train,y_test,x_test,criterion,max_depth):
    a = datetime.datetime.now()
    clf =tree.DecisionTreeClassifier(max_depth=max_depth, criterion=criterion, random_state=1)
    clf.fit(x_train, y_train)
    y_pred= clf.predict(x_test)
    b = datetime.datetime.now()
    sure=b-a
    sure=datetime.timedelta.total_seconds(sure)
    # print('Çalışma süreleri',(b-a))                          OLUŞTURULAN GRAFİKLER İÇİN GEREKLİ BÖLÜMLER
    # #print(confusion_matrix(y_test,y_pred))
    # plot_confusion_matrix(clf,x_test,y_test)
    # plt.title('Decision Tree Confusion Matrix',fontweight='bold')
    # plt.savefig('dec.png')
    # plt.show()
    # print(classification_report(y_test,y_pred))
    return clf,sure,classification_report(y_test,y_pred,output_dict='True')
# maxdepth_time=np.zeros(10)
# method_time=np.zeros(5) #GRAFİKLER İÇİN
# x_indice=np.zeros(10)
# sure=0.0
dc_clf,sure,clr=decision_tree(x_train,y_train,y_test,x_test,'entropy',20)
# precision_gr[0]=clr.get('weighted avg').get('precision')
# accuracy_gr[0]=clr.get('accuracy')                                    #GRAFİKLER İÇİN
# recall_gr[0]=clr.get('weighted avg').get('recall')
# f1score_gr[0]=clr.get('weighted avg').get('f1-score')
# method_time[0]=sure
pickle.dump(dc_clf, open('/content/decision_tree_model.sav','wb'))

In [ ]:
def random_forest_classifier(x_train,y_train,y_test,x_test):
    a = datetime.datetime.now()  
    clf=RandomForestClassifier()
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    b = datetime.datetime.now()
    sure=datetime.timedelta.total_seconds(b-a)
    print('Çalışma süreleri',(b-a)) 
    plot_confusion_matrix(clf,x_test,y_test)
    plt.title('Random Forest Confusion Matrix',fontweight='bold')
    plt.savefig('rf.png')
    plt.show()
    #print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    return clf,sure,classification_report(y_test,y_pred,output_dict=True)
rdf_clf,sure,clr=random_forest_classifier(x_train,y_train,y_test,x_test)
# precision_gr[1]=clr.get('weighted avg').get('precision')
# accuracy_gr[1]=clr.get('accuracy')
# recall_gr[1]=clr.get('weighted avg').get('recall')
# f1score_gr[1]=clr.get('weighted avg').get('f1-score')
# method_time[1]=sure
pickle.dump(rdf_clf, open('/content/random_forest_model.sav','wb'))

In [ ]:

def bernoulli_naive_bayes(x_train,y_train,y_test,x_test):
    a=datetime.datetime.now()
    gnb=BernoulliNB()
    gnb.fit(x_train,y_train)
    y_pred=gnb.predict(x_test)
    b = datetime.datetime.now()
    sure=datetime.timedelta.total_seconds(b-a)
    # print('Çalışma süreleri',(b-a)) 
    # plot_confusion_matrix(gnb,x_test,y_test)
    # plt.title('Bernoulli NB Confusion Matrix',fontweight='bold')
    # plt.savefig('bnb.png')
    # plt.show()
    # #print(confusion_matrix(y_test,y_pred))
    # print(classification_report(y_test,y_pred))
    return gnb,sure,classification_report(y_test,y_pred,output_dict='True')
bnb_clf,sure,clr=bernoulli_naive_bayes(x_train,y_train,y_test,x_test)

# precision_gr[2]=clr.get('weighted avg').get('precision')
# accuracy_gr[2]=clr.get('accuracy')
# recall_gr[2]=clr.get('weighted avg').get('recall')
# f1score_gr[2]=clr.get('weighted avg').get('f1-score')
# method_time[2]=sure
pickle.dump(bnb_clf, open('/content/bernoulli_model.sav','wb'))

In [ ]:
def naive_bayes(x_train,y_train,y_test,x_test):
    a=datetime.datetime.now()
    nb=GaussianNB()
    nb.fit(x_train,y_train)
    y_pred=nb.predict(x_test)
    b = datetime.datetime.now()
    sure=datetime.timedelta.total_seconds(b-a)
    # print('Çalışma süreleri',(b-a))
    # plot_confusion_matrix(nb,x_test,y_test)
    # plt.title('Gauss NB Confusion Matrix',fontweight='bold')
    # plt.savefig('gnb.png')
    # plt.show() 
    # #print(confusion_matrix(y_test,y_pred))
    # print(classification_report(y_test,y_pred))
    return nb,sure,classification_report(y_test,y_pred,output_dict=True)
nb_clf,sure,clr=naive_bayes(x_train,y_train,y_test,x_test)
# precision_gr[3]=clr.get('weighted avg').get('precision')
# accuracy_gr[3]=clr.get('accuracy')
# recall_gr[3]=clr.get('weighted avg').get('recall')
# f1score_gr[3]=clr.get('weighted avg').get('f1-score')
# method_time[3]=sure
pickle.dump(nb_clf, open('/content/gauss_naive_model.sav','wb'))

In [ ]:
data1=veriseti_duzenleme_ortalama()
target='Severity'
Y=data1[target]
X=data1.drop(target,axis=1)
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
def multi_naive_bayes(x_train,y_train,x_test,y_test):
    a = datetime.datetime.now()  
    clf=MultinomialNB()
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    b = datetime.datetime.now()
    sure=datetime.timedelta.total_seconds(b-a)
    print('Çalışma süreleri',(b-a)) 
    plot_confusion_matrix(clf,x_test,y_test)
    plt.title('Multinomial NB Confusion Matrix',fontweight='bold')
    plt.savefig('mnb.png')
    plt.show()
    #print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    return clf,sure,classification_report(y_test,y_pred,output_dict=True)
mnb_clf,sure,clr=multi_naive_bayes(x_train,y_train,x_test,y_test)

# precision_gr[4]=clr.get('weighted avg').get('precision')
# accuracy_gr[4]=clr.get('accuracy')
# recall_gr[4]=clr.get('weighted avg').get('recall')
# f1score_gr[4]=clr.get('weighted avg').get('f1-score')
# method_time[4]=sure

pickle.dump(nb_clf, open('/content/multinomial_model.sav','wb'))

In [ ]:
# ARAYÜZ İÇİN KULLANACAĞIMIZ TEST HAVUZUNU OLUŞTURMA
select_severity1 = data.loc[df['Severity'] == 1]
select_severity2 = data.loc[df['Severity'] == 2]
select_severity3 = data.loc[df['Severity'] == 3]
select_severity4 = data.loc[df['Severity'] == 4]
severity1=select_severity1.sample(250)
severity2=select_severity2.sample(250)
severity3=select_severity3.sample(250)
severity4=select_severity4.sample(250)
test_havuzu=pd.concat([severity1, severity2,severity3,severity4], ignore_index=True)
test_havuzu['TMC']=test_havuzu['TMC'].astype(int)
test_havuzu = test_havuzu.sample(frac=1).reset_index(drop=True)
test_havuzu.to_csv(r'test_verisi.csv',index=False)

In [ ]:
from matplotlib.figure import Figure
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from PyQt5 import QtCore, QtWidgets
import pandas as pd
import os
from PyQt5.QtWidgets import QLineEdit
from PyQt5.QtWidgets import QLabel
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
pd.options.display.float_format = '{:,.1f}'.format
matplotlib.use("Qt5Agg")


df = pd.DataFrame(None)
sns.set()


class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(1103, 640)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.label_state = QLabel('State', self.centralwidget)
        self.label_state.move(70, 20)
        self.label_state = QLabel('Year', self.centralwidget)
        self.label_state.move(240, 20)
        self.label_state = QLabel('Month', self.centralwidget)
        self.label_state.move(400, 20)
        self.label_state = QLabel('Day', self.centralwidget)
        self.label_state.move(560, 20)
        self.label_state = QLabel('Hour', self.centralwidget)
        self.label_state.move(720, 20)
        self.label_state = QLabel('Weekday', self.centralwidget)
        self.label_state.move(860, 20)
        self.state = QLineEdit(self.centralwidget)
        self.state.move(40, 40)
        self.state.resize(100, 30)
        self.centralwidget.setObjectName("centralwidget")
        self.year = QLineEdit(self.centralwidget)
        self.year.move(200, 40)
        self.year.resize(100, 30)
        self.centralwidget.setObjectName("centralwidget")
        self.month = QLineEdit(self.centralwidget)
        self.month.move(360, 40)
        self.month.resize(100, 30)
        self.centralwidget.setObjectName("centralwidget")
        self.day = QLineEdit(self.centralwidget)
        self.day.move(520, 40)
        self.day.resize(100, 30)
        self.centralwidget.setObjectName("centralwidget")
        self.hour = QLineEdit(self.centralwidget)
        self.hour.move(680, 40)
        self.hour.resize(100, 30)
        self.centralwidget.setObjectName("centralwidget")
        self.weekday = QLineEdit(self.centralwidget)
        self.weekday.move(840, 40)
        self.weekday.resize(100, 30)
        self.pushButton_1 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_1.setGeometry(QtCore.QRect(80, 80, 180, 25))
        self.pushButton_1.setObjectName("pushButton_1")
        self.pushButton_1.setEnabled(False)
        self.pushButton_2 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_2.setGeometry(QtCore.QRect(280, 80, 180, 25))
        self.pushButton_2.setObjectName("pushButton_2")
        self.pushButton_2.setEnabled(False)
        self.pushButton_3 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_3.setGeometry(QtCore.QRect(480, 80, 180, 25))
        self.pushButton_3.setObjectName("pushButton_3")
        self.pushButton_3.setEnabled(False)
        self.pushButton_4 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_4.setGeometry(QtCore.QRect(680, 80, 180, 25))
        self.pushButton_4.setObjectName("pushButton_4")
        self.pushButton_4.setEnabled(False)
        self.pushButton_5 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_5.setGeometry(QtCore.QRect(880, 80, 180, 25))
        self.pushButton_5.setObjectName("pushButton_5")
        self.pushButton_5.setEnabled(False)
        self.pushButton_6 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_6.setGeometry(QtCore.QRect(80, 180, 180, 25))
        self.pushButton_6.setObjectName("pushButton_6")
        self.pushButton_6.setEnabled(False)
        self.pushButton_7 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_7.setGeometry(QtCore.QRect(280, 180, 180, 25))
        self.pushButton_7.setObjectName("pushButton_7")
        self.pushButton_7.setEnabled(False)
        self.pushButton_8 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_8.setGeometry(QtCore.QRect(480, 180, 180, 25))
        self.pushButton_8.setObjectName("pushButton_8")
        self.pushButton_8.setEnabled(False)
        self.pushButton_9 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_9.setGeometry(QtCore.QRect(680, 180, 180, 25))
        self.pushButton_9.setObjectName("pushButton_9")
        self.pushButton_9.setEnabled(False)
        self.pushButton_10 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_10.setGeometry(QtCore.QRect(880, 180, 180, 25))
        self.pushButton_10.setObjectName("pushButton_10")
        self.pushButton_10.setEnabled(False)
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 1103, 21))
        self.menubar.setObjectName("menubar")
        self.menuFile = QtWidgets.QMenu(self.menubar)
        self.menuFile.setObjectName("menuFile")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)
        self.actionOpen_File = QtWidgets.QAction(MainWindow)
        self.actionOpen_File.setObjectName("actionOpen_File")
        self.actionExit = QtWidgets.QAction(MainWindow)
        self.actionExit.setObjectName("actionExit")
        self.menuFile.addAction(self.actionOpen_File)
        self.actionOpen_File.triggered.connect(self.readCsv)
        self.menuFile.addAction(self.actionExit)
        self.actionExit.triggered.connect(QtWidgets.QApplication.quit)
        self.menubar.addAction(self.menuFile.menuAction())
        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Deep Learning"))
        self.pushButton_1.setText(_translate(
            "MainWindow", "Decision Tree Algorithm"))
        self.pushButton_2.setText(_translate(
            "MainWindow", "Random Forest Algorithm"))
        self.pushButton_3.setText(_translate(
            "MainWindow", "Gauss Naive Bayes Algorithm"))
        self.pushButton_4.setText(_translate(
            "MainWindow", "Bernoulli Naive Bayes"))
        self.pushButton_5.setText(_translate(
            "MainWindow", "Multinomial Naive Bayes"))
        self.pushButton_6.setText(_translate(
            "MainWindow", "Multinomial Naive Bayes"))
        self.pushButton_7.setText(_translate(
            "MainWindow", "Bernoulli Naive Bayes"))
        self.pushButton_8.setText(_translate(
            "MainWindow", "Decision Tree Algorithm"))
        self.pushButton_9.setText(_translate(
            "MainWindow", "Random Forest Algorithm"))
        self.pushButton_10.setText(_translate(
            "MainWindow", "Naive Bayes Algorithm"))
        self.menuFile.setTitle(_translate("MainWindow", "File"))
        self.actionOpen_File.setText(_translate("MainWindow", "Open File"))
        self.actionExit.setText(_translate("MainWindow", "Exit"))

    def readCsv(self):
        name = QtWidgets.QFileDialog.getOpenFileName()
        file = ''.join(name)
        filename = os.path.basename(file).replace('All Files (*)', '')
        if '.csv' in filename:
            data = pd.read_csv(filename)
            target = 'Severity'
            y = data[target]
            x = data.drop(target, axis=1)
            gnbclf = pickle.load(open('gauss_naive_model.sav', 'rb'))
            dtclf = pickle.load(
                open('decision_tree_model.sav', 'rb'))
            rfclf = pickle.load(
                open('random_forest_model.sav', 'rb'))
            bnbclf = pickle.load(
                open('bernoulli_model.sav', 'rb'))
            mnbclf = pickle.load(
                open('multinomial_model.sav', 'rb'))
            gnbclf = GaussianNB()
            gnbclf.fit(x, y)
            self.msg()
            self.pushButton_1.setEnabled(True)
            self.pushButton_2.setEnabled(True)
            self.pushButton_3.setEnabled(True)
            self.pushButton_4.setEnabled(True)
            self.pushButton_5.setEnabled(True)
            self.pushButton_6.setEnabled(True)
            self.pushButton_7.setEnabled(True)
            self.pushButton_8.setEnabled(True)
            self.pushButton_9.setEnabled(True)
            self.pushButton_10.setEnabled(True)
            self.pushButton_1.clicked.connect(self.decision_tree2)
            self.pushButton_2.clicked.connect(self.randomforest2)
            self.pushButton_3.clicked.connect(self.naivebayes2)
            self.pushButton_4.clicked.connect(self.bernouli2)
            self.pushButton_5.clicked.connect(self.multinomial2)
            self.pushButton_6.clicked.connect(self.multinomial)
            self.pushButton_7.clicked.connect(self.bernouli)
            self.pushButton_8.clicked.connect(self.decision_tree)
            self.pushButton_9.clicked.connect(self.randomforest)
            self.pushButton_10.clicked.connect(self.naivebayes)
            return x, y, dtclf, rfclf, gnbclf, bnbclf, mnbclf
            # return x, y, gnbclf
        else:
            self.errorInFormat()
            return 0, 0, 0, 0, 0, 0, 0

    def msg(self):
        msg = QtWidgets.QMessageBox()
        msg.setText("Eğitim tamamlandı")
        msg.exec()

    def errorInFormat(self):
        msg = QtWidgets.QMessageBox()
        msg.setIcon(QtWidgets.QMessageBox.Critical)
        msg.setText(
            "Error with format of file. Please make sure that you've choosen .csv format file!")
        msg.setWindowTitle('FORMAT ERROR')
        msg.exec()

    def girdi_alma(self):
        state = self.state.text()
        if state in state_etiket:
            year = self.year.text()
            month = self.month.text()
            day = self.day.text()
            hour = self.hour.text()
            weekday = self.weekday.text()
            if(state.isdigit() == False and year.isdigit() and month.isdigit() and day.isdigit() and hour.isdigit() and weekday.isdigit()):
                year = int(year)
                month = int(month)
                day = int(day)
                hour = int(hour)
                weekday = int(weekday)
                stateno = state_etiket[state]
                girdi = x.loc[(x['State'] == stateno) & (x['Month'] == month) & (x['Year'] == year) & (
                    x['Day'] == day) & (x['Hour'] == hour) & (x['Weekday'] == weekday)]
                return girdi
            else:
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Critical)
                msg.setText(
                    "Yanlış Giriş Yaptınız")
                msg.setWindowTitle('TYPE ERROR')
                msg.exec()
        else:
            msg = QtWidgets.QMessageBox()
            msg.setIcon(QtWidgets.QMessageBox.Critical)
            msg.setText(
                "Eyalet Bulunmamaktadır")
            msg.setWindowTitle('TYPE ERROR')
            msg.exec()

    def bernouli2(self):
        girdi = self.girdi_alma()
        if isinstance(girdi, pd.DataFrame) == True:
            if(girdi.empty == False):
                msg = QtWidgets.QMessageBox()
                msg.setText(str(girdi.T))
                msg.setWindowTitle('Verimiz')
                msg.exec()
                tahmin = bnbclf.predict(girdi)
                gercek = y.loc[girdi.index]
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Information)
                msg.setStyleSheet("QLabel{min-width: 140px;}")
                msg.setText(
                    "Gerçek Değer:" + str(gercek.iloc[0]) + "\n" + "Tahmini değer :" + str(tahmin[0]))
                if(tahmin == 1):
                    msg.setInformativeText(
                        "Bu mevcut kaza bilgilerine göre bu kazanın trafiğe büyük bir etkisi bulunmamaktadır.")
                elif tahmin == 2:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre bu kazanın trafik yoğunluğunu arttırabilecek etkisi görülebilir. En uygun zamanda ilgili birimler tarafından müdahale edilmelidir.")
                elif tahmin == 3:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre bu kaza trafiği oldukça aksatmaktadır. Yaralanmalar da mevcut olabilir. İlgili birimlere haber verilmeli ve kaza bölgesi temizlenmelidir")
                else:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre kaza trafiği ciddi bir şekilde etkilemektedir. Acil olarak ilgili birimlere haber verilmelidir. Ciddi yaralanmalar olabilir. Bölgenin vakit kaybetmeden trafiğe açılması gerekmektedir.")
                msg.setWindowTitle('Çıktılar')
                msg.exec()
            else:
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Critical)
                msg.setText(
                    "Böyle Bir Veri Test Havuzunda Bulunmamaktadır")
                msg.setWindowTitle('INPUT ERROR')
                msg.exec()

    def multinomial2(self):
        girdi = self.girdi_alma()
        if isinstance(girdi, pd.DataFrame) == True:
            if(girdi.empty == False):
                msg = QtWidgets.QMessageBox()
                msg.setText(str(girdi.T))
                msg.setWindowTitle('Verimiz')
                msg.exec()
                tahmin = mnbclf.predict(girdi)
                gercek = y.loc[girdi.index]
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Information)
                msg.setText(
                    "Gerçek Değer:" + str(gercek.iloc[0]) + "\n" + "Tahmini değer :" + str(tahmin[0]))
                if(tahmin == 1):
                    msg.setInformativeText(
                        "Bu mevcut kaza bilgilerine göre bu kazanın trafiğe büyük bir etkisi bulunmamaktadır.")
                elif tahmin == 2:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre bu kazanın trafik yoğunluğunu arttırabilecek etkisi görülebilir. En uygun zamanda ilgili birimler tarafından müdahale edilmelidir.")
                elif tahmin == 3:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre bu kaza trafiği oldukça aksatmaktadır. Yaralanmalar da mevcut olabilir. İlgili birimlere haber verilmeli ve kaza bölgesi temizlenmelidir")
                else:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre kaza trafiği ciddi bir şekilde etkilemektedir. Acil olarak ilgili birimlere haber verilmelidir. Ciddi yaralanmalar olabilir. Bölgenin vakit kaybetmeden trafiğe açılması gerekmektedir.")
                msg.setWindowTitle('Çıktılar')
                msg.exec()
            else:
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Critical)
                msg.setText(
                    "Böyle Bir Veri Test Havuzunda Bulunmamaktadır")
                msg.setWindowTitle('INPUT ERROR')
                msg.exec()

    def naivebayes2(self):
        girdi = self.girdi_alma()
        if isinstance(girdi, pd.DataFrame) == True:
            if(girdi.empty == False):
                msg = QtWidgets.QMessageBox()
                msg.setText(str(girdi.T))
                msg.setWindowTitle('Verimiz')
                msg.exec()
                tahmin = gnbclf.predict(girdi)
                gercek = y.loc[girdi.index]
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Information)
                msg.setText(
                    "Gerçek Değer:" + str(gercek.iloc[0]) + "\n" + "Tahmini değer :" + str(tahmin[0]))
                if(tahmin == 1):
                    msg.setInformativeText(
                        "Bu mevcut kaza bilgilerine göre bu kazanın trafiğe büyük bir etkisi bulunmamaktadır.")
                elif tahmin == 2:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre bu kazanın trafik yoğunluğunu arttırabilecek etkisi görülebilir. En uygun zamanda ilgili birimler tarafından müdahale edilmelidir.")
                elif tahmin == 3:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre bu kaza trafiği oldukça aksatmaktadır. Yaralanmalar da mevcut olabilir. İlgili birimlere haber verilmeli ve kaza bölgesi temizlenmelidir")
                else:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre kaza trafiği ciddi bir şekilde etkilemektedir. Acil olarak ilgili birimlere haber verilmelidir. Ciddi yaralanmalar olabilir. Bölgenin vakit kaybetmeden trafiğe açılması gerekmektedir.")
                msg.setWindowTitle('Çıktılar')
                msg.exec()
            else:
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Critical)
                msg.setText(
                    "Böyle Bir Veri Test Havuzunda Bulunmamaktadır")
                msg.setWindowTitle('INPUT ERROR')
                msg.exec()

    def decision_tree2(self):
        girdi = self.girdi_alma()
        if isinstance(girdi, pd.DataFrame) == True:
            if(girdi.empty == False):
                msg = QtWidgets.QMessageBox()
                msg.setText(str(girdi.T))
                msg.setWindowTitle('Verimiz')
                msg.exec()
                tahmin = dtclf.predict(girdi)
                gercek = y.loc[girdi.index]
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Information)
                msg.setText(
                    "Gerçek Değer:" + str(gercek.iloc[0]) + "\n" + "Tahmini değer :" + str(int(tahmin[0])))
                if(tahmin == 1):
                    msg.setInformativeText(
                        "Bu mevcut kaza bilgilerine göre bu kazanın trafiğe büyük bir etkisi bulunmamaktadır.")
                elif tahmin == 2:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre bu kazanın trafik yoğunluğunu arttırabilecek etkisi görülebilir. En uygun zamanda ilgili birimler tarafından müdahale edilmelidir.")
                elif tahmin == 3:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre bu kaza trafiği oldukça aksatmaktadır. Yaralanmalar da mevcut olabilir. İlgili birimlere haber verilmeli ve kaza bölgesi temizlenmelidir")
                else:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre kaza trafiği ciddi bir şekilde etkilemektedir. Acil olarak ilgili birimlere haber verilmelidir. Ciddi yaralanmalar olabilir. Bölgenin vakit kaybetmeden trafiğe açılması gerekmektedir.")
                msg.setWindowTitle('Çıktılar')
                msg.exec()
            else:
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Critical)
                msg.setText(
                    "Böyle Bir Veri Test Havuzunda Bulunmamaktadır")
                msg.setWindowTitle('INPUT ERROR')
                msg.exec()

    def randomforest2(self):
        girdi = self.girdi_alma()
        if isinstance(girdi, pd.DataFrame) == True:
            if(girdi.empty == False):
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Information)
                msg.setText(str(girdi.T))
                msg.setWindowTitle('Verimiz')
                msg.exec()
                tahmin = rfclf.predict(girdi)
                gercek = y.loc[girdi.index]
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Information)
                msg.setText(
                    "Gerçek Değer:" + str(gercek.iloc[0]) + "\n" + "Tahmini değer :" + str(tahmin[0]))
                if(tahmin == 1):
                    msg.setInformativeText(
                        "Bu mevcut kaza bilgilerine göre bu kazanın trafiğe büyük bir etkisi bulunmamaktadır.")
                elif tahmin == 2:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre bu kazanın trafik yoğunluğunu arttırabilecek etkisi görülebilir. En uygun zamanda ilgili birimler tarafından müdahale edilmelidir.")
                elif tahmin == 3:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre bu kaza trafiği oldukça aksatmaktadır. Yaralanmalar da mevcut olabilir. İlgili birimlere haber verilmeli ve kaza bölgesi temizlenmelidir")
                else:
                    msg.setInformativeText(
                        "Mevcut kaza bilgilerine göre kaza trafiği ciddi bir şekilde etkilemektedir. Acil olarak ilgili birimlere haber verilmelidir. Ciddi yaralanmalar olabilir. Bölgenin vakit kaybetmeden trafiğe açılması gerekmektedir.")
                msg.setWindowTitle('Çıktılar')
                msg.exec()
            else:
                msg = QtWidgets.QMessageBox()
                msg.setIcon(QtWidgets.QMessageBox.Critical)
                msg.setText(
                    "Böyle Bir Veri Test Havuzunda Bulunmamaktadır")
                msg.setWindowTitle('INPUT ERROR')
                msg.exec()

    def naivebayes(self):
        a = np.zeros(shape=(50, 3))
        skor = 0
        for i in range(0, 50):
            z = randint(0, 999)
            deneme = x.iloc[z]
            gercek_sonuc = y.iloc[z]
            deneme = deneme.values.reshape(1, 35)
            y_pred = gnbclf.predict(deneme)
            if gercek_sonuc == y_pred:
                skor += 1*2
            a[i, 0] = int(gercek_sonuc)
            a[i, 1] = int(y_pred)
            a[i, 2] = z
        fig, ax = plt.subplots(figsize=(5, 12))
        fig.patch.set_visible(False)
        ax.axis('off')
        ax.axis('tight')
        dx = pd.DataFrame(a, columns=['Gercek Deger', 'Tahmin Deger', 'Axis'])
        dx['Gercek Deger'] = dx['Gercek Deger'].astype(np.int64)
        dx['Tahmin Deger'] = dx['Tahmin Deger'].astype(np.int64)
        dx['Axis'] = dx['Axis'].astype(np.int64)
        ax.table(cellText=dx.values, colLabels=dx.columns, loc='center')
        fig.tight_layout()
        plt.show()
        msg = QtWidgets.QMessageBox()
        msg.setIcon(QtWidgets.QMessageBox.Information)
        msg.setText(
            "Başarı Taahmin Yüzdesi : %"+str(skor))
        msg.setWindowTitle('SKOR')
        msg.exec()

    def bernouli(self):
        a = np.zeros(shape=(50, 3))
        skor = 0
        for i in range(0, 50):
            z = randint(0, 999)
            deneme = x.iloc[z]
            gercek_sonuc = y.iloc[z]
            deneme = deneme.values.reshape(1, 35)
            y_pred = bnbclf.predict(deneme)
            a[i, 0] = int(gercek_sonuc)
            a[i, 1] = int(y_pred)
            a[i, 2] = z
            if gercek_sonuc == y_pred:
                skor += 1*2
        fig, ax = plt.subplots(figsize=(5, 12))
        fig.patch.set_visible(False)
        ax.axis('off')
        ax.axis('tight')
        dx = pd.DataFrame(a, columns=['Gercek Deger', 'Tahmin Deger', 'Axis'])
        dx['Gercek Deger'] = dx['Gercek Deger'].astype(np.int64)
        dx['Tahmin Deger'] = dx['Tahmin Deger'].astype(np.int64)
        dx['Axis'] = dx['Axis'].astype(np.int64)
        ax.table(cellText=dx.values, colLabels=dx.columns, loc='center')
        fig.tight_layout()
        plt.show()
        msg = QtWidgets.QMessageBox()
        msg.setIcon(QtWidgets.QMessageBox.Information)
        msg.setText(
            "Başarı Taahmin Yüzdesi : %"+str(skor))
        msg.setWindowTitle('SKOR')
        msg.exec()

    def multinomial(self):
        a = np.zeros(shape=(50, 3))
        skor = 0
        for i in range(0, 50):
            z = randint(0, 999)
            deneme = x.iloc[z]
            gercek_sonuc = y.iloc[z]
            deneme = deneme.values.reshape(1, 35)
            y_pred = mnbclf.predict(deneme)
            a[i, 0] = int(gercek_sonuc)
            a[i, 1] = int(y_pred)
            a[i, 2] = z
            if gercek_sonuc == y_pred:
                skor += 1*2
        fig, ax = plt.subplots(figsize=(5, 12))
        fig.patch.set_visible(False)
        ax.axis('off')
        ax.axis('tight')
        dx = pd.DataFrame(a, columns=['Gercek Deger', 'Tahmin Deger', 'Axis'])
        dx['Gercek Deger'] = dx['Gercek Deger'].astype(np.int64)
        dx['Tahmin Deger'] = dx['Tahmin Deger'].astype(np.int64)
        dx['Axis'] = dx['Axis'].astype(np.int64)
        ax.table(cellText=dx.values, colLabels=dx.columns, loc='center')
        fig.tight_layout()
        plt.show()
        msg = QtWidgets.QMessageBox()
        msg.setIcon(QtWidgets.QMessageBox.Information)
        msg.setText(
            "Başarı Taahmin Yüzdesi : %"+str(skor))
        msg.setWindowTitle('SKOR')
        msg.exec()

    def decision_tree(self):
        a = np.zeros(shape=(50, 3))
        skor = 0
        for i in range(0, 50):
            z = randint(0, 999)
            deneme = x.iloc[z]
            gercek_sonuc = y.iloc[z]
            deneme = deneme.values.reshape(1, 35)
            y_pred = dtclf.predict(deneme)
            a[i, 0] = int(gercek_sonuc)
            a[i, 1] = int(y_pred)
            a[i, 2] = z
            if gercek_sonuc == y_pred:
                skor += 1*2
        fig, ax = plt.subplots(figsize=(5, 12))
        fig.patch.set_visible(False)
        ax.axis('off')
        ax.axis('tight')
        dx = pd.DataFrame(a, columns=['Gercek Deger', 'Tahmin Deger', 'Axis'])
        dx['Gercek Deger'] = dx['Gercek Deger'].astype(np.int64)
        dx['Tahmin Deger'] = dx['Tahmin Deger'].astype(np.int64)
        dx['Axis'] = dx['Axis'].astype(np.int64)
        ax.table(cellText=dx.values, colLabels=dx.columns, loc='center')
        fig.tight_layout()
        plt.show()
        msg = QtWidgets.QMessageBox()
        msg.setIcon(QtWidgets.QMessageBox.Information)
        msg.setText(
            "Başarı Taahmin Yüzdesi : %"+str(skor))
        msg.setWindowTitle('SKOR')
        msg.exec()

    def randomforest(self):
        a = np.zeros(shape=(50, 3))
        skor = 0
        for i in range(0, 50):
            z = randint(0, 999)
            deneme = x.iloc[z]
            gercek_sonuc = y.iloc[z]
            deneme = deneme.values.reshape(1, 35)
            y_pred = rfclf.predict(deneme)
            a[i, 0] = int(gercek_sonuc)
            a[i, 1] = int(y_pred)
            a[i, 2] = z
            if gercek_sonuc == y_pred:
                skor += 1*2
        fig, ax = plt.subplots(figsize=(5, 12))
        fig.patch.set_visible(False)
        ax.axis('off')
        ax.axis('tight')
        dx = pd.DataFrame(a, columns=['Gercek Deger', 'Tahmin Deger', 'Axis'])
        dx['Gercek Deger'] = dx['Gercek Deger'].astype(np.int64)
        dx['Tahmin Deger'] = dx['Tahmin Deger'].astype(np.int64)
        dx['Axis'] = dx['Axis'].astype(np.int64)
        ax.table(cellText=dx.values, colLabels=dx.columns, loc='center')
        fig.tight_layout()
        plt.show()
        msg = QtWidgets.QMessageBox()
        msg.setIcon(QtWidgets.QMessageBox.Information)
        msg.setText(
            "Başarı Taahmin Yüzdesi : %"+str(skor))
        msg.setWindowTitle('SKOR')
        msg.exec()


if __name__ == "__main__":
    state_etiket = {"OH": 33, "WV": 47, "CA": 3, "FL": 8, "GA": 9, "SC": 38, "NE": 27,
                    "IA": 10, "IL": 12, "MO": 22, "WI": 46, "IN": 13, "MI": 20, "NJ": 29, "NY": 32, "CT": 5, "MA": 17, "RI": 37, "NH": 28, "PA": 36, "KY": 15,
                    "MD": 18, "VA": 43, "DC": 6, "DE": 7, "TX": 41, "WA": 45, "OR": 35, "AL": 0, "TN": 40, "NC": 25, "KS": 14, "LA": 16, "OK": 34, "CO": 4,
                    "UT": 42, "AZ": 2, "MN": 21, "MS": 23, "NV": 31, "ME": 19, "AR": 1, "ID": 11, "VT": 44, "NM": 30, "ND": 26, "WY": 48, "SD": 39, "MT": 24}

    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    x, y, dtclf, rfclf, gnbclf, bnbclf, mnbclf = ui.readCsv()
    sys.exit(app.exec_())
